In [0]:
storage_account_name = "strgacc2831"
storage_account_key = "nK93TB+8Q47M4LiV45af/QN9W01RMS1NJYTe3ITBfE/QetSi7jp2jAtmC/DsNenMGt5aNqGjDBxR+AStA8o5SA=="

# Configure Spark to use the storage account key
spark.conf.set(f"fs.azure.account.key.strgacc2831.dfs.core.windows.net","nK93TB+8Q47M4LiV45af/QN9W01RMS1NJYTe3ITBfE/QetSi7jp2jAtmC/DsNenMGt5aNqGjDBxR+AStA8o5SA==")

In [0]:
# Set up configuration for accessing ADLS with a SAS token
spark.conf.set(
    "fs.azure.sas.silver.strgacc2831.dfs.core.windows.net",
    "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-11-08T09:12:09Z&st=2024-11-08T01:12:09Z&spr=https&sig=93v7nCVWLv3xH6v%2Fx5KvVAloWG83JJNJ5vBX9dcA%2Bjw%3D"
)


In [0]:
# Assuming both files are in CSV format
accounts_df = spark.read.format("csv").option("header", "true").load("abfss://silver@strgacc2831.dfs.core.windows.net/cleaned_accounts.csv/part-00000-tid-1480919646493366281-e08f047d-1733-4059-8af7-4a1ba5f7a417-42-1-c000.csv")
customers_df = spark.read.format("csv").option("header", "true").load("abfss://silver@strgacc2831.dfs.core.windows.net/customers.csv")

accounts_df.show(5)
customers_df.show(5)




+-----------+----------+------------+-------+
|customer_id|account_id|account_type|balance|
+-----------+----------+------------+-------+
|          7|        28|    Checking|2900.00|
|         51|        41|     Savings| 250.25|
|         15|        38|    Checking|3900.50|
|         54|        89|     Savings| 850.25|
|         11|        24|    Checking|2600.00|
+-----------+----------+------------+-------+
only showing top 5 rows

+-----------+----------+---------+--------------+---------+-----+------+
|customer_id|first_name|last_name|       address|     city|state|   zip|
+-----------+----------+---------+--------------+---------+-----+------+
|          1|      John|      Doe|    123 Elm St|  Toronto|   ON|M4B1B3|
|          2|      Jane|    Smith| 456 Maple Ave|   Ottawa|   ON|K1A0B1|
|          3|   Michael|  Johnson|    789 Oak Dr| Montreal|   QC|H1A1A1|
|          4|     Emily|    Davis|   101 Pine Rd|  Calgary|   AB|T2A0A1|
|          5|     David|   Wilson|202 Birch Blvd|V

In [0]:
from pyspark.sql import functions as F

# Convert balance column to float if necessary
accounts_df = accounts_df.withColumn("balance", F.col("balance").cast("float"))

# Join customers and accounts DataFrames on customer_id and select specific columns to avoid ambiguity
joined_df = customers_df.alias("cust").join(accounts_df.alias("acct"), F.col("cust.customer_id") == F.col("acct.customer_id"), "inner") \
    .select(
        F.col("cust.customer_id").alias("customer_id"),
        "first_name",
        "last_name",
        "address",
        "city",
        "state",
        "zip",
        "account_type",
        "balance"
    )

# Aggregate to calculate the total balance for each customer
result_df = joined_df.groupBy(
    "customer_id",
    "first_name",
    "last_name",
    "address",
    "city",
    "state",
    "zip",
    "account_type"
).agg(
    F.sum("balance").alias("total_balance")
)
# Display the result
result_df.show()

+-----------+-----------+---------+---------------+---------------+-----+------+------------+-------------+
|customer_id| first_name|last_name|        address|           city|state|   zip|account_type|total_balance|
+-----------+-----------+---------+---------------+---------------+-----+------+------------+-------------+
|         86|     Olivia|   Gibson|    8585 Elm St|   New Liskeard|   ON|P0J0A1|     Savings|       400.25|
|         81|    Michael|    Owens| 8080 Willow Rd|        Mattawa|   ON|P0H0A1|    Checking|       2700.0|
|         26|    Abigail|   Parker| 2525 Poplar St|         Barrie|   ON|L4M0A1|    Checking|       6700.5|
|         20|        Mia|   Nelson|1919 Birch Blvd|         London|   ON|N6A0A1|    Checking|       6100.0|
|          5|      David|   Wilson| 202 Birch Blvd|      Vancouver|   BC|V5K0A1|    Checking|       1600.5|
|         53|      James|  Jenkins| 5252 Willow Rd|    Queensville|   ON|L0G0A1|     Savings|       300.25|
|         83|      David|   

In [0]:
# Write the sorted DataFrame to Azure Data Lake Storage in CSV format
result_df.write.format("csv").mode("overwrite").option("header", "true").save("abfss://silver@strgacc2831.dfs.core.windows.net/total_balance_per_customer.csv")

In [0]:
# Assuming `total_balance_df` is the DataFrame with the aggregated total balances
gold_container_path = "abfss://gold@strgacc2831.dfs.core.windows.net/total_balance_per_customer.csv"

result_df.write.format("csv").mode("overwrite").option("header", "true").save("abfss://gold@strgacc2831.dfs.core.windows.net/total_balance_per_customer.csv")

result_df.show()

+-----------+-----------+---------+---------------+---------------+-----+------+------------+-------------+
|customer_id| first_name|last_name|        address|           city|state|   zip|account_type|total_balance|
+-----------+-----------+---------+---------------+---------------+-----+------+------------+-------------+
|         86|     Olivia|   Gibson|    8585 Elm St|   New Liskeard|   ON|P0J0A1|     Savings|       400.25|
|         81|    Michael|    Owens| 8080 Willow Rd|        Mattawa|   ON|P0H0A1|    Checking|       2700.0|
|         26|    Abigail|   Parker| 2525 Poplar St|         Barrie|   ON|L4M0A1|    Checking|       6700.5|
|         20|        Mia|   Nelson|1919 Birch Blvd|         London|   ON|N6A0A1|    Checking|       6100.0|
|          5|      David|   Wilson| 202 Birch Blvd|      Vancouver|   BC|V5K0A1|    Checking|       1600.5|
|         53|      James|  Jenkins| 5252 Willow Rd|    Queensville|   ON|L0G0A1|     Savings|       300.25|
|         83|      David|   